In [2]:
import pandas as pd
from datetime import datetime
import networkx as nx

In [3]:
data = pd.read_csv('output.csv')
test_data= pd.read_csv('test_output.csv')

In [6]:
import openai
import time

# Read the OpenAI API key from a file
with open('openAiKey.txt', 'r') as file:
    api_key = file.read().strip()

# Initialize the OpenAI client
client = openai.OpenAI(api_key=api_key)

# Fine-tuning job ID (replace with your actual fine-tuning job ID)
fine_tune_job_id = "ftjob-7KYfEjLJRIIzjxM0sgnaxp3g"

# Function to check the status of the fine-tuning job and retrieve the model ID
def check_fine_tune_status(job_id):
    status_response = client.fine_tuning.jobs.retrieve(job_id)
    return status_response

# Monitor the fine-tuning job status
status = check_fine_tune_status(fine_tune_job_id)
print(f"Initial Status: {status.status}")

# Poll the status until the job is complete
while status.status not in ['succeeded', 'failed']:
    time.sleep(60)  # Wait for 60 seconds before checking the status again
    status = check_fine_tune_status(fine_tune_job_id)
    print(f"Current Status: {status.status}")

# Check the final status and retrieve model details if successful
if status.status == 'succeeded':
    fine_tuned_model = status.fine_tuned_model
    print(f"Fine-tuning succeeded. Fine-tuned Model: {fine_tuned_model}")
else:
    print("Fine-tuning failed. Please check the details and logs for more information.")


Initial Status: succeeded
Fine-tuning succeeded. Fine-tuned Model: ft:gpt-3.5-turbo-0125:personal::9XQB3irt


In [7]:
import openai

# Read the OpenAI API key from a file
with open('openAiKey.txt', 'r') as file:
    api_key = file.read().strip()

# Initialize the OpenAI client
client = openai.OpenAI(api_key=api_key)

# Example usage of the fine-tuned chat model
def predict_issue_resolution_time(issue_text, model):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an assistant that helps predict the time to resolve GitHub issues."},
            {"role": "user", "content": issue_text}
        ],
        max_tokens=10  # Adjust as needed
    )
    return response.choices[0].message.content.strip()

# Prepare the test issues for prediction
test_issues = test_data.apply(lambda row: f"Title: {row['Issue Title']}\nBody: {row['Issue Body']}\nLabels: {row['Labels']}\nNumber of Comments: {row['Number of Comments']}\nComment Bodies: {row['Comment Bodies']}\nSocial Metrics: wordiness={row['wordiness']}, average_degree_centrality={row['average_degree_centrality']}, average_closeness={row['average_closeness']}, average_betweenness={row['average_betweenness']}, density={row['density']}, edges={row['edges']}, num_discussants={row['num_discussants']}\n", axis=1)

# Use the correct fine-tuned model ID
fine_tuned_model = "ft:gpt-3.5-turbo-0125:personal::9XQB3irt"  # Replace with your actual fine-tuned model ID

# Predict and validate on the test data
test_data['Predicted Issue Lifetime'] = test_issues.apply(lambda issue_text: predict_issue_resolution_time(issue_text, fine_tuned_model))

# Print the predictions
print(test_data[['Issue Title', 'Issue Lifetime', 'Predicted Issue Lifetime']])


                                           Issue Title  Issue Lifetime  \
0                                sync footer with news             NaN   
1               codepen signup link leads to 404 error             NaN   
2    react write a react component from scratch tes...             NaN   
3                           overflow hidden in firefox             NaN   
4                            refactor menu to use refs             NaN   
..                                                 ...             ...   
593  fix tests for strict order in step 25 of oop p...             NaN   
594  missleading comments in js exercise declare a ...             NaN   
595                                post userreportuser             NaN   
596                  update learn footer based on news             NaN   
597     browser frozen on fibonacci lost weeks of work             NaN   

    Predicted Issue Lifetime  
0                   < 2 days  
1       2 days < x < 2 weeks  
2                 

In [8]:
# Function to categorize issue lifetime numerically
def categorize_lifetime(hours):
    try:
        hours = float(hours)
    except ValueError:
        return None
    if hours < 48:
        return 1  # <2 days
    elif hours < 336:  # 2 weeks = 14 days * 24 hours = 336 hours
        return 2  # 2 days < x < 2 weeks
    else:
        return 3  # >2 weeks

# Apply the categorization to actual and predicted lifetimes
test_data['Issue Lifetime Bin'] = test_data['Issue Lifetime (hours)'].apply(categorize_lifetime)


In [9]:
# Function to categorize issue lifetime from words
def categorize_lifetime_words(bin):
    if bin =="<2 days":
        return 1  # <2 days
    elif bin =="2 days < x < 2 weeks":  # 2 weeks = 14 days * 24 hours = 336 hours
        return 2  # 2 days < x < 2 weeks
    else:
        return 3  # >2 weeks
    

# Apply the categorization to actual and predicted lifetimes
test_data['Predicted Issue Lifetime Bin'] = test_data['Predicted Issue Lifetime'].apply(categorize_lifetime_words)

In [10]:
# Drop rows where categorization failed (None values)
test_data = test_data.dropna(subset=['Issue Lifetime Bin', 'Predicted Issue Lifetime Bin'])

# Check Data Distribution
print("Actual Issue Lifetime Bin Distribution:")
print(test_data['Issue Lifetime Bin'].value_counts())

print("Predicted Issue Lifetime Bin Distribution:")
print(test_data['Predicted Issue Lifetime Bin'].value_counts())

# Display some of the actual and predicted values
print("Sample Actual and Predicted Issue Lifetime Bins:")
print(test_data[['Issue Lifetime Bin', 'Predicted Issue Lifetime Bin']].head(10))

# Test the categorization function with known values
print("Categorization Function Test:")
print(categorize_lifetime(24))  # Expected output: 1 (<2 days)
print(categorize_lifetime(100))  # Expected output: 2 (2 days < x < 2 weeks)
print(categorize_lifetime(500))  # Expected output: 3 (>2 weeks)

Actual Issue Lifetime Bin Distribution:
Issue Lifetime Bin
3    252
2    182
1    164
Name: count, dtype: int64
Predicted Issue Lifetime Bin Distribution:
Predicted Issue Lifetime Bin
3    377
2    221
Name: count, dtype: int64
Sample Actual and Predicted Issue Lifetime Bins:
   Issue Lifetime Bin  Predicted Issue Lifetime Bin
0                   1                             3
1                   1                             2
2                   2                             3
3                   1                             2
4                   2                             2
5                   1                             2
6                   3                             3
7                   1                             2
8                   2                             2
9                   3                             2
Categorization Function Test:
1
2
3


In [11]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, f1_score, precision_score, recall_score

# Calculate weighted average F1-score, precision, and recall
labels = [1, 2, 3]
f1 = f1_score(test_data['Issue Lifetime Bin'], test_data['Predicted Issue Lifetime Bin'], labels=labels, average='weighted')
precision = precision_score(test_data['Issue Lifetime Bin'], test_data['Predicted Issue Lifetime Bin'], labels=labels, average='weighted')
recall = recall_score(test_data['Issue Lifetime Bin'], test_data['Predicted Issue Lifetime Bin'], labels=labels, average='weighted')

# Calculate confusion matrix
cm = confusion_matrix(test_data['Issue Lifetime Bin'], test_data['Predicted Issue Lifetime Bin'], labels=labels)

cm_df = pd.DataFrame(cm, index=labels, columns=labels)

/Users/atu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# Calculate TP, FP, FN, TN
results = {}
for i, label in enumerate(labels):
    results[label] = {'TP': cm[i, i]}
    results[label]['FP'] = cm[:, i].sum() - cm[i, i]
    results[label]['FN'] = cm[i, :].sum() - cm[i, i]
    results[label]['TN'] = cm.sum() - (results[label]['TP'] + results[label]['FP'] + results[label]['FN'])

# Print results
for label, metrics in results.items():
    print(f"{label}: {metrics}")

# Save results to CSV
results_df = pd.DataFrame(results).T
results_df['F1-score'] = f1
results_df['Recall'] = recall
results_df['Precision'] = precision


1: {'TP': 0, 'FP': 0, 'FN': 164, 'TN': 434}
2: {'TP': 88, 'FP': 133, 'FN': 94, 'TN': 283}
3: {'TP': 172, 'FP': 205, 'FN': 80, 'TN': 141}


In [13]:
# Print classification report
print(classification_report(test_data['Issue Lifetime Bin'], test_data['Predicted Issue Lifetime Bin'], target_names=['<2 days', '2 days < x < 2 weeks', '>2 weeks']))

                      precision    recall  f1-score   support

             <2 days       0.00      0.00      0.00       164
2 days < x < 2 weeks       0.40      0.48      0.44       182
            >2 weeks       0.46      0.68      0.55       252

            accuracy                           0.43       598
           macro avg       0.28      0.39      0.33       598
        weighted avg       0.31      0.43      0.36       598



/Users/atu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/atu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/atu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
